# Diffusion Music Generation Level 2

### imports

In [ ]:
import IPython

import torch
from torch.utils.data import DataLoader
import yaml
import matplotlib.pyplot as plt
import torchaudio

from models.multi_level_vqvae import MultiLvlVQVariationalAutoEncoder
from models.diffusion_vit import DiffusionViTSongCond
from models.diffusion_unet import WaveNetDiffusion
from models.transformer_vqvae import TransformerVQVAE
from loaders.latent_loaders import Lvl2InputDataset, Lvl3InputDataset
from utils.other import load_cfg_dict

### Load configuration files and add weight paths

In [ ]:
# Level 1
config_path_lvl1 = "config/lvl1_config.yaml"
weights_path_lvl1 = "model_weights/lvl1_vqvae.ckpt"
cfg_1 = load_cfg_dict(config_path_lvl1)

config_path_diff_lvl1 = "config/diff_lvl1_config.yaml"
weights_path_diff_lvl1 = "model_weights/lvl1_diff.ckpt"
cfg_diff_1 = load_cfg_dict(config_path_diff_lvl1)

# Level 2
config_path_lvl2 = "config/lvl2_config.yaml"
weights_path_lvl2 = "model_weights/lvl2_vqvae.ckpt"
cfg_2 = load_cfg_dict(config_path_lvl2)

config_path_diff_lvl2 = "config/diff_lvl2_config.yaml"
weights_path_diff_lvl2 = "model_weights/lvl2_diff.ckpt"
cfg_diff_2 = load_cfg_dict(config_path_diff_lvl2)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Load the models

In [ ]:
# Level 1
model_vqvae_lvl1 = MultiLvlVQVariationalAutoEncoder.load_from_checkpoint(weights_path_lvl1, **cfg_1, strict=False).to(device=device)
model_vqvae_lvl1.eval()

model_diffusion_lvl1 = DiffusionViTSongCond.load_from_checkpoint(weights_path_diff_lvl1, **cfg_diff_1, strict=False).to(device=device)
model_diffusion_lvl1.eval()

# Level 2
model_vqvae_lvl2 = MultiLvlVQVariationalAutoEncoder.load_from_checkpoint(weights_path_lvl2, **cfg_2, strict=False).to(device=device)
model_vqvae_lvl2.eval()

model_diffusion_lvl2 = DiffusionViTSongCond.load_from_checkpoint(weights_path_diff_lvl2, **cfg_diff_2, strict=False).to(device=device)
model_diffusion_lvl2.eval()

### Define the player

In [ ]:
# this is a wrapper that take a filename and publish an html <audio> tag to listen to it

def wavPlayer(filepath):
    """ will display html 5 player for compatible browser

    Parameters :
    ------------
    filepath : relative filepath with respect to the notebook directory ( where the .ipynb are not cwd)
               of the file to play

    The browser need to know how to play wav through html5.

    there is no autoplay to prevent file playing when the browser opens
    """
    
    src = """
    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <title>Simple Test</title>
    </head>
    
    <body>
    <audio controls="controls" style="width:600px" >
      <source src="files/%s" type="audio/mp3" />
      Your browser does not support the audio element.
    </audio>
    </body>
    """%(filepath)
    display(HTML(src))

## Load the diffusion dataset and display the latents

In [ ]:
dataset = Lvl3InputDataset(preload=True, **cfg_diff_2)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

for sample in dataloader:
    lvl3_sample = sample['music slice'].squeeze(0)
    print(f"Current track is {sample['track name'][0]}")
    break

plt.figure(figsize=(25, 5))
plt.matshow(lvl3_sample.cpu().detach().numpy(), fignum=1, aspect='auto', vmin=-5, vmax=5.0)
plt.colorbar()
plt.axis('off')
plt.show()

## Run diffusion model level 2

In [ ]:
random_noise_start = torch.randn_like(lvl3_sample)
lvl2_latent_gen = model_diffusion_lvl2.denoise(lvl3_sample.unsqueeze(0).to(device), show_process_plots=False).squeeze(0)

plt.figure(figsize=(25, 5))
plt.matshow(lvl2_latent_gen.cpu().detach().numpy(), fignum=1, aspect='auto', vmin=-5, vmax=5.0)
plt.colorbar()
plt.axis('off')
plt.show()

## Generate level 1 latent

In [ ]:
with torch.no_grad():
    lvl1_latent, _ = model_vqvae_lvl2.decode(lvl3_sample.unsqueeze(0).to(device))
plt.figure(figsize=(25, 5))
plt.matshow(lvl1_latent[:, :].squeeze(0).cpu().detach().numpy(), fignum=1, aspect='auto', vmin=-5, vmax=5.0)
plt.colorbar()
plt.axis('off')
plt.show()

## Run VQVAE decoding level 1

In [ ]:
from torch.utils.data import TensorDataset

batch_size = 4
batch_divide = 8

with torch.no_grad():
    
    lvl3_sample *= 0.0
    lvl3_sample += torch.randn_like(lvl3_sample) * 1.5
    
    plt.figure(figsize=(25, 5))
    plt.matshow(lvl3_sample[:, :].squeeze(0).cpu().detach().numpy(), fignum=1, aspect='auto', vmin=-5, vmax=5.0)
    plt.colorbar()
    plt.axis('off')
    plt.show()

    
    output_lvl1 = torch.zeros([1, cfg_1['input_channels'], 0]).to(device=device)

    diff_lvl3_sample = model_diffusion_lvl2.denoise(lvl3_sample.unsqueeze(0).to(device))

    output_lvl2, _ = model_vqvae_lvl2.decode(diff_lvl3_sample)
    output_lvl2 = output_lvl2.permute((0, 2, 1)).reshape((batch_divide, -1, cfg_1['latent_depth'])).permute((0, 2, 1))
    
    lvl2_dataset = TensorDataset(output_lvl2)
    lvl2_dataloader = DataLoader(lvl2_dataset, batch_size=batch_size)
    
    for lvl2_ind_sample in lvl2_dataloader:
        
        # diff_lvl2_sample = model_diffusion_lvl1.denoise(lvl2_ind_sample[0].to(device))
        diff_lvl2_sample = lvl2_ind_sample[0].to(device)
        
        output_lvl1_ind, _ = model_vqvae_lvl1.decode(diff_lvl2_sample)
        output_lvl1_ind = output_lvl1_ind.permute((0, 2, 1)).reshape((1, -1, 1)).permute((0, 2, 1))
        output_lvl1 = torch.cat((output_lvl1, output_lvl1_ind), dim=2)
            
music_sample_rec = output_lvl1.view((1, -1))
plt.figure(figsize=(25, 5))
plt.plot(music_sample_rec[0, ...].cpu().detach().numpy())
plt.ylim((-1.1, 1.1))
plt.show()
torchaudio.save('sample_out.mp3', music_sample_rec.cpu().detach(), 44100, format='mp3')
IPython.display.Audio(filename="sample_out.mp3")